# Calendar methods

This tutorial offers a walk through of all the ExchangeCalendar methods (for properties see the [calendar properties](./calendar_properties.ipynb) tutorial).

The following sections cover methods according to the nature of the argument(s) they take.

* [Methods that query a Date](#Methods-that-query-a-Date)  
* [Methods that query a Session](#Methods-that-query-a-Session)  
* [Methods that query a Minute](#Methods-that-query-a-Minute)  
* [Methods that query multiple TradingMinute](#Methods-that-query-multiple-TradingMinute)  
* [Methods that query a Date/Session range](#Methods-that-query-a-Date/Session-range)  

The following sections cover methods that evaluate an index of TradingMinute or Session.
* [Methods that evaluate an index of contiguous TradingMinute](#Methods-that-evaluate-an-index-of-contiguous-TradingMinute)  
* [Methods that evaluate an index of contiguous Session](#Methods-that-evaluate-an-index-of-contiguous-Session)  

`Date` and `Session` refer to types for 'session' parameters (see [sessions.ipynb](./sessions.ipynb) for a tutorial on how to work with sessions).  
`Minute` and `TradingMinute` refer to types for 'minute' parameters (see [minutes.ipynb](./minutes.ipynb) for a tutorial on how to work with mintues).

In [2]:
# setup
import exchange_calendars as xcals
import pandas as pd

one_minute = pd.Timedelta(1, "T")

xnys = xcals.get_calendar("XNYS", side="left") # New York Stock Exchange
xhkg = xcals.get_calendar("XHKG", side="left") # Hong Kong Stock Exchange

### Methods that query a Date

The parameter of methods in this section takes a `Date` type.

In [3]:
# for reference, all times are UTC
xnys.schedule.loc["2020-12-31":"2021-01-05"]

,market_open,break_start,break_end,market_close
2020-12-31 00:00:00+00:00,2020-12-31 14:30:00,NaT,NaT,2020-12-31 21:00:00
2021-01-04 00:00:00+00:00,2021-01-04 14:30:00,NaT,NaT,2021-01-04 21:00:00
2021-01-05 00:00:00+00:00,2021-01-05 14:30:00,NaT,NaT,2021-01-05 21:00:00


In [4]:
# does a date represent a session
xnys.is_session("2021-01-01"), xnys.is_session("2021-01-05")

(False, True)

**`date_to_session`** will return a session for the passed `date` if `date` represents a session, or otherwise the closest session in the passed `direction`.

In [5]:
xnys.date_to_session_label("2021-01-01", direction="next")

Timestamp('2021-01-04 00:00:00+0000', tz='UTC', freq='C')

In [6]:
xnys.date_to_session_label("2021-01-01", direction="previous")

Timestamp('2020-12-31 00:00:00+0000', tz='UTC', freq='C')

In [ ]:
# ValueError raised if `date` is not a session and no `direction` is passed
xnys.date_to_session_label("2021-01-01")
# run cell for full traceback

```python
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_9148/2744539978.py in <module>
----> 1 xnys.date_to_session_label("2021-01-01")

ValueError: `date` '2021-01-01 00:00:00+00:00' is not a session label. Consider passing a `direction`.
```

In [8]:
# date is a session, so `direction` ignored
xnys.date_to_session_label("2021-01-05", direction="next")

Timestamp('2021-01-05 00:00:00+0000', tz='UTC')

### Methods that query a Session

The parameter of methods in this section takes a `Session` type.

The following methods can be used to return the **open, close** and **break times** of a session...

In [9]:
# for reference, all times are UTC
session = "2021-01-04"
xhkg.schedule.loc[[session]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 01:30:00,2021-01-04 04:00:00,2021-01-04 05:00:00,2021-01-04 08:00:00


In [10]:
xhkg.session_open(session), xhkg.session_close(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 08:00:00+0000', tz='UTC'))

In [11]:
# or
xhkg.open_and_close_for_session(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 08:00:00+0000', tz='UTC'))

In [12]:
xhkg.session_break_start(session), xhkg.session_break_end(session)

(Timestamp('2021-01-04 04:00:00+0000', tz='UTC'),
 Timestamp('2021-01-04 05:00:00+0000', tz='UTC'))

In [13]:
# or
xhkg.break_start_and_end_for_session(session)

(Timestamp('2021-01-04 04:00:00+0000', tz='UTC'),
 Timestamp('2021-01-04 05:00:00+0000', tz='UTC'))

In [14]:
# in local time
xhkg.session_open(session).tz_convert(xhkg.tz)

Timestamp('2021-01-04 09:30:00+0800', tz='Asia/Hong_Kong')

The **`session_*_minute`** methods return a boundary trading minute for a session. 

In [15]:
xhkg.session_first_minute(session), xhkg.session_last_minute(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 07:59:00+0000', tz='UTC'))

In [16]:
xhkg.session_last_am_minute(session), xhkg.session_first_pm_minute(session)

(Timestamp('2021-01-04 03:59:00+0000', tz='UTC'),
 Timestamp('2021-01-04 05:00:00+0000', tz='UTC'))

In [17]:
# or get first and last minutes together...
xhkg.session_first_and_last_minute(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 07:59:00+0000', tz='UTC'))

In [18]:
# NB with "right" side...
xhkg_right = xcals.get_calendar("XHKG", side="right")
xhkg_right.session_first_and_last_minute(session)

(Timestamp('2021-01-04 01:31:00+0000', tz='UTC'),
 Timestamp('2021-01-04 08:00:00+0000', tz='UTC'))

**`session_has_break`** to query if a session has a break.

In [19]:
# Hong Kong has a break (at least on `session`)...
xhkg.session_has_break(session)

True

In [20]:
# but New York doesn't
xnys.session_has_break(session)

False

In [21]:
# see...
xnys.schedule.loc[[session]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 14:30:00,NaT,NaT,2021-01-04 21:00:00


**`next_session`** and **`previous_session`** do what they say on the tin.

In [22]:
# just to recall...
session

'2021-01-04'

In [23]:
xhkg.previous_session_label(session)

Timestamp('2020-12-31 00:00:00+0000', tz='UTC', freq='C')

In [24]:
xhkg.next_session_label(session)

Timestamp('2021-01-05 00:00:00+0000', tz='UTC', freq='C')

**`minutes_for_session`** is a cracker - returns an index of all the trading minutes of the given session

In [25]:
xhkg.minutes_for_session(session)

DatetimeIndex(['2021-01-04 01:30:00+00:00', '2021-01-04 01:31:00+00:00',
               '2021-01-04 01:32:00+00:00', '2021-01-04 01:33:00+00:00',
               '2021-01-04 01:34:00+00:00', '2021-01-04 01:35:00+00:00',
               '2021-01-04 01:36:00+00:00', '2021-01-04 01:37:00+00:00',
               '2021-01-04 01:38:00+00:00', '2021-01-04 01:39:00+00:00',
               ...
               '2021-01-04 07:50:00+00:00', '2021-01-04 07:51:00+00:00',
               '2021-01-04 07:52:00+00:00', '2021-01-04 07:53:00+00:00',
               '2021-01-04 07:54:00+00:00', '2021-01-04 07:55:00+00:00',
               '2021-01-04 07:56:00+00:00', '2021-01-04 07:57:00+00:00',
               '2021-01-04 07:58:00+00:00', '2021-01-04 07:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=330, freq=None)

### Methods that query a Minute

The parameter of methods in this section takes a `Minute` type.

**`is_trading_minute`** returns a boolean indicating if a minute is a trading minute.

In [26]:
# for reference
xhkg.schedule.loc[["2021-01-04"]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 01:30:00,2021-01-04 04:00:00,2021-01-04 05:00:00,2021-01-04 08:00:00


In [27]:
xhkg.is_trading_minute("2021-01-04 01:25")

False

In [28]:
xhkg.is_trading_minute("2021-01-04 01:35")

True

In [29]:
break_minute = "2021-01-04 04:35"
xhkg.is_trading_minute(break_minute)

False

**`is_break_minute`** returns a boolean indicating if a minute lies within a lunch break. Break minutes lie between the last trading minute of the morning subsession and first trading minute of the afternoon subsession, exclusive of both (a break minute cannot be a trading minute).

In [30]:
xhkg.is_break_minute(break_minute)

True

**`is_open_on_minute`** can take an optional `include_breaks` argument. If `include_breaks` is False (the default) then the method behaves in the same way as `is_trading_minute`. If `include_breaks` is True then the method will return True if `minute` is either a trading minute or a break minute (otherwise returns False).

In [31]:
xhkg.is_open_on_minute(break_minute)

False

In [32]:
xhkg.is_open_on_minute(break_minute, ignore_breaks=True)

True

**`previous_open`**, **`next_open`**, **`previous_close`** and **`next_close`** will return the previous or next open or close relative to the passed `minute`. NB Which minutes are treated as trading minutes is irrelevant.

In [33]:
# for reference
xnys.schedule.loc["2021-01-04":"2021-01-05"]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 14:30:00,NaT,NaT,2021-01-04 21:00:00
2021-01-05 00:00:00+00:00,2021-01-05 14:30:00,NaT,NaT,2021-01-05 21:00:00


In [34]:
open_05 = xnys.session_open("2021-01-05")

In [35]:
xnys.previous_open(open_05), xnys.next_open(open_05)

(Timestamp('2021-01-04 14:30:00+0000', tz='UTC'),
 Timestamp('2021-01-06 14:30:00+0000', tz='UTC'))

In [36]:
xnys.previous_open(open_05 + one_minute)

Timestamp('2021-01-05 14:30:00+0000', tz='UTC')

In [37]:
xnys.next_close(open_05)

Timestamp('2021-01-05 21:00:00+0000', tz='UTC')

**`previous_minute`** and **`next_minute`** return the first trading minute prior to / following `minute`.

In [38]:
minute = xnys.all_first_minutes["2021-01-05"]
minute

Timestamp('2021-01-05 14:30:00+0000', tz='UTC')

In [39]:
xnys.previous_minute(minute), xnys.next_minute(minute)

(Timestamp('2021-01-04 20:59:00+0000', tz='UTC'),
 Timestamp('2021-01-05 14:31:00+0000', tz='UTC'))

In [40]:
minute -= (one_minute * 2)
minute

Timestamp('2021-01-05 14:28:00+0000', tz='UTC')

In [41]:
xnys.previous_minute(minute), xnys.next_minute(minute)

(Timestamp('2021-01-04 20:59:00+0000', tz='UTC'),
 Timestamp('2021-01-05 14:30:00+0000', tz='UTC'))

**`minute_to_session`** returns the session associated with `minute`. If `minute` is a trading minute or a break minute then this will be the session of which the minute is a trading/break minute.

In [42]:
trading_minute = xnys.all_minutes[233]  # get a trading minute
trading_minute

Timestamp('2001-09-24 17:23:00+0000', tz='UTC')

In [43]:
xnys.minute_to_session_label(trading_minute)

Timestamp('2001-09-24 00:00:00+0000', tz='UTC', freq='C')

If `minute` is not a trading or break minute then the return is determined by the optional `direction` parameter. If `direction` is "next" the return will be the first session after `minute`, whilst if `direction` is "previous" the return will be the first session before `minute`.

In [44]:
# for reference
minute, xnys.is_trading_minute(minute)

(Timestamp('2021-01-05 14:28:00+0000', tz='UTC'), False)

In [45]:
# default `direction` is "next"
xnys.minute_to_session_label(minute)

Timestamp('2021-01-05 00:00:00+0000', tz='UTC', freq='C')

In [46]:
xnys.minute_to_session_label(minute, direction="previous")

Timestamp('2021-01-04 00:00:00+0000', tz='UTC', freq='C')

`direction` can also take "none", in which case `minute` is required to be a trading or break minute. To the contrary a ValueError is raised:

In [ ]:
xnys.minute_to_session_label(minute, direction="none")
# run cell for full traceback

```python
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_9148/525547029.py in <module>
----> 1 xnys.minute_to_session_label(minute, direction="none")

ValueError: Received `minute` as '2021-01-05 14:29:00+00:00' although this is not an exchange minute. Consider passing `direction` as 'next' or 'previous'.
```

### Methods that evaluate an index of contiguous TradingMinute

**`minutes_in_range`** returns all trading minutes within and inclusive of `start_minute` and `end_minute`. The parameters take `Minute` type (i.e.they do not need to represent actual trading minutes).

In [48]:
# recalling...
xhkg.schedule.loc[["2021-01-04"]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 01:30:00,2021-01-04 04:00:00,2021-01-04 05:00:00,2021-01-04 08:00:00


In [49]:
xhkg.minutes_in_range("2021-01-04 03:55", "2021-01-04 05:05")

DatetimeIndex(['2021-01-04 03:55:00+00:00', '2021-01-04 03:56:00+00:00',
               '2021-01-04 03:57:00+00:00', '2021-01-04 03:58:00+00:00',
               '2021-01-04 03:59:00+00:00', '2021-01-04 05:00:00+00:00',
               '2021-01-04 05:01:00+00:00', '2021-01-04 05:02:00+00:00',
               '2021-01-04 05:03:00+00:00', '2021-01-04 05:04:00+00:00',
               '2021-01-04 05:05:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [50]:
# `start_minute` does not have to represent a trading minute
xhkg.minutes_in_range("2021-01-04 04:30", "2021-01-04 05:05")

DatetimeIndex(['2021-01-04 05:00:00+00:00', '2021-01-04 05:01:00+00:00',
               '2021-01-04 05:02:00+00:00', '2021-01-04 05:03:00+00:00',
               '2021-01-04 05:04:00+00:00', '2021-01-04 05:05:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Alternatively, **`minutes_window`** can be used to create an index of TradingMinute of a defined length, anchored by the given TradingMinute.

In [51]:
xhkg.minutes_window("2021-01-04 07:55", count=9)

DatetimeIndex(['2021-01-04 07:55:00+00:00', '2021-01-04 07:56:00+00:00',
               '2021-01-04 07:57:00+00:00', '2021-01-04 07:58:00+00:00',
               '2021-01-04 07:59:00+00:00', '2021-01-05 01:30:00+00:00',
               '2021-01-05 01:31:00+00:00', '2021-01-05 01:32:00+00:00',
               '2021-01-05 01:33:00+00:00', '2021-01-05 01:34:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Or if you want, work backwards from `minute`...

In [52]:
xhkg.minutes_window("2021-01-04 07:55", count=-9)

DatetimeIndex(['2021-01-04 07:46:00+00:00', '2021-01-04 07:47:00+00:00',
               '2021-01-04 07:48:00+00:00', '2021-01-04 07:49:00+00:00',
               '2021-01-04 07:50:00+00:00', '2021-01-04 07:51:00+00:00',
               '2021-01-04 07:52:00+00:00', '2021-01-04 07:53:00+00:00',
               '2021-01-04 07:54:00+00:00', '2021-01-04 07:55:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

**NB** The index will have length one greater than the `count` (this behaviour may change in release 4.0, see [#61](https://github.com/gerrymanoim/exchange_calendars/issues/61)).

### Methods that query multiple TradingMinute

**`minutes_to_sessions`** does for multiple TradingMinute what `minute_to_session` does for one. However, this method is limited by requiring every minute of `minutes` to be a trading minute (i.e. there's no `direction` parameter available for this one). Also, whilst minutes do not need to be contiguous, they do need to be ordered ascendingly.

In [53]:
# set up a minute index..
ser = xnys.all_first_minutes[-200:-195] + (one_minute * 5)
ser

2021-12-08 00:00:00+00:00   2021-12-08 14:35:00+00:00
2021-12-09 00:00:00+00:00   2021-12-09 14:35:00+00:00
2021-12-10 00:00:00+00:00   2021-12-10 14:35:00+00:00
2021-12-13 00:00:00+00:00   2021-12-13 14:35:00+00:00
2021-12-14 00:00:00+00:00   2021-12-14 14:35:00+00:00
Freq: C, Name: first_minutes, dtype: datetime64[ns, UTC]

In [54]:
mins = pd.DatetimeIndex(ser)
mins = mins[:1].union(mins[-2:], sort=False)
mins

DatetimeIndex(['2021-12-08 14:35:00+00:00', '2021-12-13 14:35:00+00:00',
               '2021-12-14 14:35:00+00:00'],
              dtype='datetime64[ns, UTC]', name='first_minutes', freq=None)

In [55]:
xnys.minute_index_to_session_labels(mins)

DatetimeIndex(['2021-12-08 00:00:00+00:00', '2021-12-13 00:00:00+00:00',
               '2021-12-14 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [56]:
# NB `minutes` can also be defined as a Series...
xnys.minute_index_to_session_labels(ser)

DatetimeIndex(['2021-12-08 00:00:00+00:00', '2021-12-09 00:00:00+00:00',
               '2021-12-10 00:00:00+00:00', '2021-12-13 00:00:00+00:00',
               '2021-12-14 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### Methods that evaluate an index of contiguous Session

**`sessions_in_range`** does for sessions what `minutes_in_range` does for minutes. Returns an index of all the sessions between `start_session` and `end_session`. Parameters take `Date` type (i.e. do not need to represent actual sessions).

In [57]:
xnys.sessions_in_range("2020", "2020-12-31")

DatetimeIndex(['2020-01-02 00:00:00+00:00', '2020-01-03 00:00:00+00:00',
               '2020-01-06 00:00:00+00:00', '2020-01-07 00:00:00+00:00',
               '2020-01-08 00:00:00+00:00', '2020-01-09 00:00:00+00:00',
               '2020-01-10 00:00:00+00:00', '2020-01-13 00:00:00+00:00',
               '2020-01-14 00:00:00+00:00', '2020-01-15 00:00:00+00:00',
               ...
               '2020-12-17 00:00:00+00:00', '2020-12-18 00:00:00+00:00',
               '2020-12-21 00:00:00+00:00', '2020-12-22 00:00:00+00:00',
               '2020-12-23 00:00:00+00:00', '2020-12-24 00:00:00+00:00',
               '2020-12-28 00:00:00+00:00', '2020-12-29 00:00:00+00:00',
               '2020-12-30 00:00:00+00:00', '2020-12-31 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=253, freq='C')

**`sessions_window`** holds no surprises if you're familiar with `minutes_window`...

In [58]:
xnys.sessions_window("2020-12-23", 3)

DatetimeIndex(['2020-12-23 00:00:00+00:00', '2020-12-24 00:00:00+00:00',
               '2020-12-28 00:00:00+00:00', '2020-12-29 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='C')

In [59]:
xnys.sessions_window("2020-12-23", -3)

DatetimeIndex(['2020-12-18 00:00:00+00:00', '2020-12-21 00:00:00+00:00',
               '2020-12-22 00:00:00+00:00', '2020-12-23 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='C')

**NB** As for `minutes_window`, the length of the returned index is one greater than `count` (...and again this may change with release 4.0, see [#61](https://github.com/gerrymanoim/exchange_calendars/issues/61))

### Methods that query a Date/Session range

The methods in this section query a range of sessions from `start_session` through `end_session` (inclusive of both). Both parameters take a `Date` (i.e they can but do not have to represent a session).

In [60]:
start_session, end_session = "2021-12-23", "2021-12-29"
# for reference
xnys.schedule.loc[start_session:end_session]

,market_open,break_start,break_end,market_close
2021-12-23 00:00:00+00:00,2021-12-23 14:30:00,NaT,NaT,2021-12-23 21:00:00
2021-12-27 00:00:00+00:00,2021-12-27 14:30:00,NaT,NaT,2021-12-27 21:00:00
2021-12-28 00:00:00+00:00,2021-12-28 14:30:00,NaT,NaT,2021-12-28 21:00:00
2021-12-29 00:00:00+00:00,2021-12-29 14:30:00,NaT,NaT,2021-12-29 21:00:00


**`sessions_distance`** returns the number of sessions in the range.

In [61]:
xnys.session_distance(start_session, end_session)

4

**`sessions_minutes`** returns an index comprised of the TradingMinutes of all sessions in the given range.

In [62]:
xnys.minutes_for_sessions_in_range(start_session, end_session)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:31:00+00:00',
               '2021-12-23 14:32:00+00:00', '2021-12-23 14:33:00+00:00',
               '2021-12-23 14:34:00+00:00', '2021-12-23 14:35:00+00:00',
               '2021-12-23 14:36:00+00:00', '2021-12-23 14:37:00+00:00',
               '2021-12-23 14:38:00+00:00', '2021-12-23 14:39:00+00:00',
               ...
               '2021-12-29 20:50:00+00:00', '2021-12-29 20:51:00+00:00',
               '2021-12-29 20:52:00+00:00', '2021-12-29 20:53:00+00:00',
               '2021-12-29 20:54:00+00:00', '2021-12-29 20:55:00+00:00',
               '2021-12-29 20:56:00+00:00', '2021-12-29 20:57:00+00:00',
               '2021-12-29 20:58:00+00:00', '2021-12-29 20:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=1560, freq=None)

**`sessions_minutes_count`** returns just the number of trading minutes corresponding to the session range.

In [63]:
xnys.minutes_count_for_sessions_in_range(start_session, end_session)

1560

In [64]:
# i.e. `sessions_minutes_count` behaves as...
len(xnys.minutes_for_sessions_in_range(start_session, end_session))

1560

**`session_opens`** and **`sessions_closes`** return pd.Series describing open/close times over the session range.

In [65]:
xnys.session_opens_in_range(start_session, end_session)

2021-12-23 00:00:00+00:00   2021-12-23 14:30:00+00:00
2021-12-27 00:00:00+00:00   2021-12-27 14:30:00+00:00
2021-12-28 00:00:00+00:00   2021-12-28 14:30:00+00:00
2021-12-29 00:00:00+00:00   2021-12-29 14:30:00+00:00
Freq: C, Name: market_open, dtype: datetime64[ns, UTC]

NB the return differs from the following only in that **`session_opens`** returns as tz "UTC"

In [66]:
xnys.opens[start_session:end_session]

2021-12-23 00:00:00+00:00   2021-12-23 14:30:00
2021-12-27 00:00:00+00:00   2021-12-27 14:30:00
2021-12-28 00:00:00+00:00   2021-12-28 14:30:00
2021-12-29 00:00:00+00:00   2021-12-29 14:30:00
Freq: C, Name: market_open, dtype: datetime64[ns]

...and following release 4.0 ([see #61](https://github.com/gerrymanoim/exchange_calendars/issues/61)) there will probably be no difference.